In [3]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import csv

In [5]:
url = 'https://en.wikipedia.org/wiki/FIFA_World_Cup'
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html.parser')

#Get Links for World Cups in Overview Table
attendance = soup.find('h2', string='Attendance')
overviewTable = attendance.find_next('table')
links = overviewTable.find_all('a')
worldCups = []
for link in links:
    href = link['href']
    if (href.endswith('FIFA_World_Cup')):
        url = 'https://en.wikipedia.org' + href
        worldCups.append(url)

# SPECIAL LOGIC 1978/1982/1986/1990/1994/1998/2006 GROUP STAGE

worldCupCount = 0
for worldCup in worldCups:
    print(worldCupCount)
    page = requests.get(worldCup)
    soup = BeautifulSoup(page.text, 'html.parser')

    #Crawl one game logic
    year = soup.find('h1', id='firstHeading').get_text()[0:4]

    games = soup.find_all('div', class_='footballbox')
    iteration = 0
    for game in games:
        teamA = game.find('th', class_='fhome').get_text().strip()
        teamB = game.find('th', class_='faway').get_text().strip()
        score = game.find('th', class_='fscore').get_text()
        print(teamA + ' ' + teamB + ' ' + score)
        overtime = False
        if 'a.e.t' in score:
            overtime = True
            score = score.split(' ')[0]
        if '–' in score:
            goalsA = score.split('–')[0]
            goalsB = score.split('–')[1]
            if (int(goalsA) > int(goalsB)):
                winner = 'teamA'
            elif (int(goalsA) == int(goalsB)):
                winner = 'draw'
                if overtime:
                    winner = 'unclear'
            elif (int(goalsA) < int(goalsB)):
                winner = 'teamB'
            else:
                winner = 'ERROR'
        else:
            goalsA = 'unclear'
            goalsB = 'unclear'

        #WRITE TO CSV
        with open('WorldCupData.csv', mode='a', newline='', encoding='utf-8') as outputFile:
            worldCupCSV = csv.writer(outputFile, delimiter=',', quotechar='"', quoting = csv.QUOTE_MINIMAL)

            if (worldCupCount == 0 and iteration == 0):
                worldCupCSV.writerow(['year', 'teamA', 'teamB', 'goalsA', 
                    'goalsB', 'winner'])
        
            worldCupCSV.writerow([year, teamA, teamB, goalsA, goalsB, winner])
        iteration = iteration + 1
    worldCupCount = worldCupCount+1

0
France Mexico 4–1
Argentina France 1–0
Chile Mexico 3–0
Chile France 1–0
Argentina Mexico 6–3
Argentina Chile 3–1
Yugoslavia Brazil 2–1
Yugoslavia Bolivia 4–0
Brazil Bolivia 4–0
Romania Peru 3–1
Uruguay Peru 1–0
Uruguay Romania 4–0
United States Belgium 3–0
United States Paraguay 3–0
Paraguay Belgium 1–0
Argentina United States 6–1
Uruguay Yugoslavia 6–1
Uruguay Argentina 4–2
1
Spain Brazil 3–1
Hungary Egypt 4–2
Switzerland Netherlands 3–2
Italy United States 7–1
Czechoslovakia Romania 2–1
Sweden Argentina 3–2
Austria France 3–2 (a.e.t.)
Germany Belgium 5–2
Austria Hungary 2–1
Italy Spain 1–1 (a.e.t.)
Germany Sweden 2–1
Czechoslovakia Switzerland 3–2
Italy Spain 1–0
Italy Austria 1–0
Czechoslovakia Germany 3–1
Germany Austria 3–2
Italy Czechoslovakia 2–1 (a.e.t.)
2
Switzerland Germany 1–1 (a.e.t.)
Hungary Dutch East Indies 6–0
Sweden Austria w/o[a]
Cuba Romania 3–3 (a.e.t.)
France Belgium 3–1
Italy Norway 2–1 (a.e.t.)
Brazil Poland 6–5 (a.e.t.)
Czechoslovakia Netherlands 3–0 (a.e.t.)